In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

The thermal conductivity equation:
$\frac{\partial T}{\partial t} - \frac{{\partial}^2 T}{\partial x^2} = 0$

Assume $ T(t, x) = 2 + e^{-4 \pi^2 t} sin(2\pi x) + e^{-16\pi^2 t} cos(4\pi x)$, $x\in[0,1], t\in[0, 0.05]$
$\\$Boundary condition $T_0 = T(0, x) = 2 + sin(2\pi x) + cos(4\pi x)$

In [2]:
#define amount of sample points
N = 2000

In [3]:
#in the beginning we plot graph of the thermal conductivity process
def f_real(t, x):
    return (2 + torch.exp(-4*(torch.pi**2)*t)*torch.sin(2*torch.pi*x) + torch.exp(-16*(torch.pi**2)*t)*torch.cos(4*torch.pi*x))

In [4]:
# Now we want to get test points from equation in first cell
x_data = torch.rand(N).view(-1,1)
t_data = 0.01 * torch.rand(N).view(-1,1)

In [5]:
#define the class PINN
class PINN(nn.Module):
    def __init__(self,input_layer=2,h1=64,h2=64,h3=64,output_layer=1):
        super().__init__()
        self.fc1 = nn.Linear(input_layer,h1)
        self.fc2 = nn.Linear(h1,h2)
        self.fc3 = nn.Linear(h2,h3)
        self.fc4 = nn.Linear(h3,output_layer)
        
    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = F.gelu(self.fc2(x))
        x = F.gelu(self.fc3(x))
        x = F.gelu(self.fc4(x))
        
        return x

In [6]:
x_phys = torch.rand(N).view(-1,1).requires_grad_(True)
t_ = 5 * torch.rand(N).view(-1,1)
t_phys = t_.requires_grad_(True)
points = torch.stack((t_phys,x_phys), -1)
points_bc = torch.stack((torch.zeros(N,1), x_phys), -1)
torch.manual_seed(41)

In [9]:
pinn = PINN()
optimizer = torch.optim.Adam(pinn.parameters(), lr=0.01)#SGD, 
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 60, gamma=0.5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.75)

Loss1 = $\frac{1}{N} \sum_{i=1}^{N}(T(t_i, x_i)- f_{PINN}(t_i, x_i))^2$

Loss2 = $\frac{1}{N} \sum_{i=1}^{N}(T_0(x_i)- f_{PINN}(0, x_i))^2$

Loss3 = $\frac{1}{N} \sum_{i=1}^{N}(\frac{\partial}{\partial t}f_{PINN}(t_i, x_i) - \frac{{\partial}^2 }{\partial x^2}f_{PINN}(t_i, x_i))^2$

In [10]:
"""
Neural network training using sum of MSE, boundary condition and partial dirivatives.
"""
epochs = 500

for i in range(epochs):
    #compute MSE of T(t,x) and points that were pridicted by PINN
    network = pinn.forward(points)
    loss1 = torch.mean((f_real(t_data, x_data) - network)**2)
    y_bc = pinn.forward(points_bc)
    loss2 = torch.mean((f_real(torch.zeros_like(x_data), x_data) - y_bc)**2)
    
    #compute loss using derivatives
    dt = torch.autograd.grad(network, t_phys, torch.ones_like(network), create_graph=True)[0]
    dx = torch.autograd.grad(network, x_phys, torch.ones_like(network), create_graph=True)[0]
    dx2 = torch.autograd.grad(dx, x_phys, torch.ones_like(dx), create_graph=True)[0]
    loss3 = torch.mean((dt - dx)**2)
    
    loss = loss1 + loss2 + loss3
    loss.backward()
    
    optimizer.step()
    scheduler.step(loss)
    if i % 50 == 0:
        print(f'epoch: {i}\tamount of loss: {loss}\t')#learning rate: {scheduler.get_last_lr()}')

epoch: 0	amount of loss: 9.264941215515137	
epoch: 50	amount of loss: 4.5181379318237305	
epoch: 100	amount of loss: 2.562826156616211	
epoch: 150	amount of loss: 1.5607805252075195	
epoch: 200	amount of loss: 1.6356854438781738	
epoch: 250	amount of loss: 1.5877504348754883	
epoch: 300	amount of loss: 1.5806422233581543	
epoch: 350	amount of loss: 1.5771487951278687	
epoch: 400	amount of loss: 1.575698971748352	
epoch: 450	amount of loss: 1.5751796960830688	


In [49]:
#MSE between neural network and given function
f_nn = pinn.forward(points)
print(torch.mean((f_real(t_data, x_data) - f_nn)**2))

tensor(0.0844, grad_fn=<MeanBackward0>)


In [52]:
"""
Neural network training using only partial dirivatives and boundary condition.
"""
pinn_new = PINN()
optimizer_new = torch.optim.Adam(pinn_new.parameters(), lr=0.01)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 60, gamma=0.5)
scheduler_new = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_new, 'min', factor=0.75)
epochs = 500

for i in range(epochs):
    #compute MSE of T(t,x) and points that were pridicted by PINN
    network_new = pinn_new.forward(points)
    y_bc_new = pinn_new.forward(points_bc)
    loss2_new = torch.mean((f_real(torch.zeros_like(x_data), x_data) - y_bc_new)**2)
    
    #compute loss using derivatives
    dt_new = torch.autograd.grad(network_new, t_phys, torch.ones_like(network_new), create_graph=True)[0]
    dx_new = torch.autograd.grad(network_new, x_phys, torch.ones_like(network_new), create_graph=True)[0]
    dx2_new = torch.autograd.grad(dx_new, x_phys, torch.ones_like(dx_new), create_graph=True)[0]
    loss3_new = torch.mean((dt_new - dx_new)**2)
    
    loss_new = loss2_new + loss3_new
    loss_new.backward()
    
    optimizer_new.step()
    scheduler_new.step(loss)
    if i % 50 == 0:
        print(f'epoch: {i}\tamount of loss: {loss_new}\t')

epoch: 0	amount of loss: 4.951014041900635	
epoch: 50	amount of loss: 1.277353286743164	
epoch: 100	amount of loss: 1.3161898851394653	
epoch: 150	amount of loss: 1.2619175910949707	
epoch: 200	amount of loss: 1.1619625091552734	
epoch: 250	amount of loss: 1.1108143329620361	
epoch: 300	amount of loss: 1.0971288681030273	
epoch: 350	amount of loss: 1.093517541885376	
epoch: 400	amount of loss: 1.0925331115722656	
epoch: 450	amount of loss: 1.0922656059265137	


In [54]:
#MSE between neural network and given function
f_nn = pinn_new.forward(points)
print(torch.mean((f_real(t_data, x_data) - f_nn)**2))

tensor(0.0958, grad_fn=<MeanBackward0>)
